Import library and *package*

In [ ]:
!wget https://github.com/ngatclc3/echocardiogram/tree/main/assets/DATA_CHAMBER_2021.zip

--2021-11-11 08:39:18--  https://github.com/ngatclc3/echocardiogram/tree/main/assets/DATA_CHAMBER_2021.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ngatclc3/echocardiogram/blob/main/assets/DATA_CHAMBER_2021.zip [following]
--2021-11-11 08:39:18--  https://github.com/ngatclc3/echocardiogram/blob/main/assets/DATA_CHAMBER_2021.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘DATA_CHAMBER_2021.zip.1’

DATA_CHAMBER_2021.z     [ <=>                ] 133.99K  --.-KB/s    in 0.1s    

2021-11-11 08:39:19 (1.18 MB/s) - ‘DATA_CHAMBER_2021.zip.1’ saved [137206]



In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn, optim
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader

from google.colab import drive
import os

from torchvision.datasets import ImageFolder

from collections import namedtuple
from sklearn.metrics import classification_report

In [ ]:
# Use pretrained models which are available in torchvision
pretrained_models = {
    'vgg16': models.vgg16(pretrained=True),
    'resnet18': models.resnet18(pretrained=True),
    'densenet': models.densenet161(pretrained=True),
    'inception': models.inception_v3(pretrained=True),
    'googlenet': models.googlenet(pretrained=True)
}

In [ ]:
# Generate different transformations for each data type
CJ = transforms.ColorJitter()
Rs = transforms.Resize(size=64)
CC = transforms.CenterCrop(size=224)
RR = transforms.RandomRotation(degrees=15)
RRC = transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0))

RC = transforms.RandomCrop(32, padding=4)
RHF = transforms.RandomHorizontalFlip()
RVF = transforms.RandomVerticalFlip()
TPI = transforms.ToPILImage()
TT = transforms.ToTensor()
Nml = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transform_train_preprocess =  transforms.Compose([CJ, RR, TT, Nml])
transform_train_rescale =  transforms.Compose([Rs, RRC, CC, TT, Nml])
transform_train_with_aug = transforms.Compose([TPI, RC, RHF, TT, Nml])
transform_train_no_aug = transforms.Compose([TT, Nml])

transform_test = transforms.Compose([TT])

In [ ]:
train_path = "/content/drive/MyDrive/ColabNotebooks/assets/DATA_CHAMBER_2021/train"
test_path = "/content/drive/MyDrive/ColabNotebooks/assets/DATA_CHAMBER_2021/test"

def prepare_data(transform_train):
  trainset = ImageFolder(root = train_path, transform = transform_train)
  testset = ImageFolder(root = test_path, transform = transform_test)
  return TrainTest(train = trainset, 
                   test = testset)

TrainTest = namedtuple('TrainTest', ['train', 'test'])

def prepare_loader(datasets):
  batch = 32
  worker = 2
  trainloader = DataLoader(dataset = datasets.train,
                           batch_size = batch, 
                           shuffle = True, 
                           num_workers = worker)
  testloader = DataLoader(dataset = datasets.test, 
                          batch_size = batch, 
                          shuffle = False, 
                          num_workers = worker)
  return TrainTest(train = trainloader, 
                   test = testloader)


In [ ]:
def get_trainer(model):
    loss = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    return loss, optimizer

def train_epoch(epoch, model, loader, device):
    model.train()
    running_loss = 0.0
    reporting_step = 30

    loss, optimizer = get_trainer(model)

    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % reporting_step == reporting_step-1:
            print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
            running_loss = 0.0
            
def test_epoch(epoch, model, loader, device):
    model.eval()
    ypred = []
    ytrue = []
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)
        ypred += list(predicted.cpu().numpy())
        ytrue += list(labels.cpu().numpy())
    return ypred, ytrue

In [ ]:
def run(model):
  datasets = prepare_data(transform_train_no_aug)
  loaders = prepare_loader(datasets)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device=device)

  accuracy = []
  classes = ['2C','3C','4C']

  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue_test, ypred_test, target_names=classes))

    # calculate accurency
    ypred_test = np.array(ypred_test)
    ytrue_test = np.array(ytrue_test)
    accuracy = (ytrue_test == ypred_test).sum() / len(ytrue_test)
    print()

In [ ]:
def main():
  for model_name, model in pretrained_models.items():
    run(model)

In [ ]:
if __name__ == "__main__":
  main()